In [1]:
from qmcpy import *
import numpy as np
import pandas as pd
import math as math
pd.options.display.float_format = '{:.2e}'.format
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
from scipy.stats import norm
from qmcpy.discrete_distribution._discrete_distribution import LD
from qmcpy.accumulate_data import MLQMCData
from qmcpy.integrand import  MLCallOptions
import time as t
from qmcpy.integrand import Integrand
from qmcpy.discrete_distribution import DigitalNetB2
from qmcpy.true_measure import Gaussian
from qmcpy.util import ParameterError, MaxSamplesWarning, ParameterError, MaxLevelsWarning
import warnings
import array
from numpy.linalg import lstsq
from qmcpy.accumulate_data import AccumulateData

In [3]:
level_dims = [2,4,8]
ac2_multilevel = AsianOption(DigitalNetB2(seed=7),multilevel_dims=level_dims)
levels_to_spawn = np.arange(ac2_multilevel.max_level+1)
ac2_single_levels = ac2_multilevel.spawn(levels_to_spawn)
yml = 0
for ac2_single_level in ac2_single_levels:
    x = ac2_single_level.discrete_distrib.gen_samples(2**12)
    level_est = ac2_single_level.f(x).mean()
    yml += level_est
yml

1.7788556123958916

In [ ]:
class FMLAsianOption(Integrand):

    def __init__(self, sampler, volatility=.2, option='asian', start_price=100.,
        strike_price=100., interest_rate=.05, t_final=1., _level=0, 
                 call_put='call', mean_type='arithmetic', multilevel_dims=[4,16,64]):
        
        self.ac2_multilevel = AsianOption(sampler, multilevel_dims=multilevel_dims, volatility=volatility, option=option, start_price=start_price,
        strike_price=strike_price, interest_rate=interest_rate, t_final=t_final, _level=_level, 
                 call_put=call_put, mean_type=mean_type)
        
        super(FMLAsianOption,self).__init__(dimension_indv=len(multilevel_dims),dimension_comb=1,parallel=False)
    
    def g(self, t, *args):
        y=np.zeros((len(t), self.t_indv))
        for k in range (self.d_indv):
            d=self.ac2_multilevel[k].true_measure.d
            y[:,k]=self.ac2_multilevel[k].g(t[:,])
            
            